In [1]:
%load_ext autoreload
%autoreload 2

In [127]:
import numpy as np
import pandas as pd
import xarray as xr
import tdt
from collections.abc import Iterable

In [5]:
xr.set_options(keep_attrs=True)

In [202]:
from pathlib import Path
block_path = Path('/Volumes/paxilline/Data/PAX_3/PAX_3_TANK/PAX_3-sal3-bl')

In [203]:
store_names = load_store_names(block_path)
channel_lists = load_channel_lists(block_path)

read from t=0s to t=86358.05s
read from t=0s to t=86358.05s


In [205]:
store_name = store_names[0]
channels = channel_lists[store_name][-2:]
print(f"Store name: {store_name}")
print(f"Channels: {channels}")

Store name: EEG_
Channels: [5, 6]


In [208]:
start_time, end_time = (0, 60)

In [209]:
def load_store_names(block_path):
    """Get the names of each stream store in the block."""
    hdr = tdt.read_block(block_path, headers=True)
    return [store.name for key, store in hdr.stores.items()]

def load_channel_lists(block_path):
    """Get the list of channels associated with each stream store."""
    hdr = tdt.read_block(block_path, headers=True)
    return {store.name: list(np.unique(store.chan)) for key, store in hdr.stores.items()}

In [218]:
def _load_stream_store(block_path, store_name, chans=None, start_time=0, end_time=0):
    """Load a single stream store from disk.
    
    Parameters:
    -----------
    block_path: 
        Path to the TDT block directory which contains files of type .Tbk, .tev, .tsq, etc. 
    store_name: string
        The name of the stream store to load.
    chans: Iterable, optional, default: None
        The list of channels to load, as they appear in Synapse/OpenEx. These will be 1-indexed, 
        rather than 0-indexed. Passing `None` (default) loads all channels. 
    start_time: float, optional, default: 0
        Start time of the data to load, relative to the file start, in seconds. 
    end_time: float, optional, default: 0
        End time of the data to load, relative to the file start, in seconds.
        Passing `0` (default) will load until the end of the file.  
    
    Returns:
    --------
    info: tdt.StructType
        The `info` field of a tdt `blk` struct, used to get the file start time.
    store: tdt.StructType
        The store field of a tdt `blk.streams` struct, which contains the data. 
    """
    assert store_name not in [
        "epocs",
        "snips",
        "streams",
        "scalars",
        "info",
        "time_ranges",
    ]

    read_block_kwargs = dict(store=["info", store_name], t1=start_time, t2=end_time)
    if chans:
        assert isinstance(chans, Iterable)
        assert 0 not in chans, "Passing 0 to tdt.read_block will load all channels."
        read_block_kwargs.update(channel=chans)

    blk = tdt.read_block(block_path, **read_block_kwargs)
    return blk.info, blk.streams[store_name]

In [211]:
info, store = _load_stream_store(block_path, store_name, chans=channels, start_time=start_time, end_time=end_time)

read from t=0s to t=5991.81s


In [212]:
def stream_store_to_xarray(info, store):
    """Convert a single stream store to xarray format.
    
    Paramters:
    ----------
    info: tdt.StructType
        The `info` field of a tdt `blk` struct, as returned by `_load_stream_store`.
    store: tdt.StructType
        The store field of a tdt `blk.streams` struct, as returned by `_load_stream_store`.
        
    Returns:
    --------
    data: xr.DataArray (n_samples, n_channels)
        Values: The data, in microvolts. 
        Attrs: units, fs
        Name: The store name
    """  
    n_channels, n_samples = store.data.shape

    time = np.arange(0, n_samples) / store.fs
    timedelta = pd.to_timedelta(time, "s")
    datetime = pd.to_datetime(info.start_date) + timedelta

    volts_to_microvolts = 1e6
    data = xr.DataArray(
        store.data.T * volts_to_microvolts,
        dims=("time", "channel"),
        coords={
            "time": time,
            "channel": store.channel,
            "timedelta": ("time", timedelta),
            "datetime": ("time", datetime),
        },
        name=store.name,
    )
    data.attrs["units"] = "uV"
    data.attrs["fs"] = store.fs

    return data

In [221]:
def load_stream_store(*args, **kwargs):
    """Load a single stream store from disk and convert to xarray format.
    
    Parameters:
    -----------
    block_path: 
        Path to the TDT block directory which contains files of type .Tbk, .tev, .tsq, etc. 
    store_name: string
        The name of the stream store to load.
    chans: Iterable, optional, default: None
        The list of channels to load, as they appear in Synapse/OpenEx. These will be 1-indexed, 
        rather than 0-indexed. Passing `None` (default) loads all channels. 
    start_time: float, optional, default: 0
        Start time of the data to load, relative to the file start, in seconds. 
    end_time: float, optional, default: 0
        End time of the data to load, relative to the file start, in seconds.
        Passing `0` (default) will load until the end of the file. 
        
    Returns:
    --------
    data: xr.DataArray (n_samples, n_channels)
        Values: The data, in microvolts. 
        Attrs: units, fs
        Name: The store name
    """  
    info, store = _load_stream_store(*args, **kwargs)
    return stream_store_to_xarray(info, store)

In [214]:
sig = load_stream_store(block_path, store_name, chans=channels, start_time=start_time, end_time=end_time)

read from t=0s to t=5991.81s


In [215]:
def load_block(block_path, start_time=0, end_time=0):
    """Load all stream stores in a block."""
    store_names = load_store_names(block_path)
    store_data = dict()
    for name in store_names:
        store_data[name] = load_stream_store(block_path, store_name, start_time=start_time, end_time=end_time)
    
    return store_data

In [216]:
d = load_block(block_path, start_time=start_time, end_time=end_time)

read from t=0s to t=86358.05s
read from t=0s to t=5991.81s
read from t=0s to t=5991.81s
read from t=0s to t=5991.81s
